# Music App

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import os
import glob
import csv
from IPython.display import display
import pandas as pd
import cassandra
from cassandra.cluster import Cluster


#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_data_new.csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_data_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow(
            (row[0], row[2], row[3], row[4], row[5], row[6], 
             row[7], row[8], row[12], row[13], row[16])
        )


In [ ]:
# check the number of rows in your csv file
with open('event_data_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## The CSV file titled <font color=red>event_data_new.csv</font>, located within the Workspace directory contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_data_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# Making a connection to a Cassandra instance in local machine, i.e., localhost or 127.0.0.1
try:
    cluster = Cluster(['localhost'])
    # Create a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [ ]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Setting the keyspace
try:
    session.set_keyspace("music_app")
except Exception as e:
    print(e)

# Create queries to ask the following three questions of the data

Let's use pandas to check the columns order in the CSV file and a few other things

In [ ]:
df = pd.read_csv('./event_data_new.csv')
df

## Query 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Description of primary key

Our choice of primary key consisted of a composite partition key (session_id, item_in_session) and no clustering column
- The composite partition key reflects the way we want to query the data, i.e., by session_id and item_in_session.
- This increases the performance of the query.

Ensuring that the primary key is unique, i.e., has no duplicates

In [ ]:
df.groupby(['sessionId', 'itemInSession']).size().max()

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
table_1 = 'song_by_session_and_item'
table_for_query_1 = f"""
create table if not exists {table_1} (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    song_length double,
    primary key ((session_id, item_in_session))   
);
"""

try:
    session.execute(table_for_query_1)
except Exception as e:
    print(e)        

#### Insert data into table_1

In [ ]:
%%time
try:
    file = 'event_data_new.csv'
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"""
            insert into {table_1} (session_id, item_in_session, artist, song, song_length)
            VALUES (%s, %s, %s, %s, %s)
            """
            session.execute(
                query, 
                (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into table_1

In [ ]:
try:
    result = pd.DataFrame(
        session
        .execute(f"""
            select artist, song, song_length
            from {table_1}
            where session_id = 338 and item_in_session = 4;
            """)
    ) 
    display(result)
except Exception as e:
    print(e)

## Query 2
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

First, note that there are no duplicates in the CSV file when we consider the columns 'userId', 'sessionId', and 'itemInSession'. In other words, they form a composite primary key for the table.

### Description of primary key

Our choice of primary key consisted of a composite partition key (user_id, session_id) and a clustering column item_in_session
- The composite partition key reflects the way we want to query the data, i.e., by user_id and session_id.
- Since we want our results ordered by item_in_session, we add this column as a clustering column.

Note that the primary key is unique, i.e., has no duplicates

In [ ]:
df.groupby(['userId', 'sessionId', 'itemInSession']).size().max()

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
table_2 = "songs_by_session"
table_for_query_2 = f"""
create table if not exists {table_2} (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    user_first_name text,
    user_last_name text,
    primary key ((user_id, session_id), item_in_session)   
);
"""

try:
    session.execute(table_for_query_2)
except Exception as e:
    print(e)        

#### Insert data into table_2

In [ ]:
%%time
try:
    file = 'event_data_new.csv'
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"""
            insert into {table_2}
                (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            session.execute(
                query, 
                (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into table 2

In [ ]:
try:
    result = pd.DataFrame(
        session
        .execute(f"""
            select artist, song, user_first_name, user_last_name
            from {table_2}
            where user_id = 10 and session_id = 182;
            """)
    ) 
    display(result)
except Exception as e:
    print(e)

## Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Description of primary key

Our choice of primary key consisted of a partition key (song) and a clustering column (user_id)
- The partition key reflects the way we want to query the data, i.e., by song.
- However, using only song as the primary key would not work, since we would have only one row for each song.
- Therefore, we added the user_id column as a clustering column.

Remark:
- It does not matter here that (song, user_id) has duplicates.
- If the same user listened to a particular song multiple times, we want to list they only once in our query result.

In [ ]:
df.groupby(['song', 'userId']).size().max()

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
table_3 = "listeners_by_song"
table_for_query_3 = f"""
create table if not exists {table_3} (
    song text,
    user_id int,
    user_first_name text,
    user_last_name text,
    primary key (song, user_id)   
);
"""

try:
    session.execute(table_for_query_3)
except Exception as e:
    print(e)        

#### Insert data into table_3

In [ ]:
%%time
try:
    file = 'event_data_new.csv'
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"""
            insert into {table_3} (song, user_id, user_first_name, user_last_name)
            VALUES (%s, %s, %s, %s)
            """
            session.execute(
                query, 
                (line[9], int(line[10]), line[1], line[4]))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into table_3

In [ ]:
try:
    result = pd.DataFrame(
        session
        .execute(f"""
            select user_first_name, user_last_name
            from {table_3}
            where song = 'All Hands Against His Own';
            """)
    ) 
    display(result)
except Exception as e:
    print(e)

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute(f'drop table if exists {table_1}')
    session.execute(f'drop table if exists {table_2}')
    session.execute(f'drop table if exists {table_3}')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()